# Doawnloads and Stores IPO S-1 Filings as .txt Files

* Uses EDGAR search
 * https://www.sec.gov/edgar/searchedgar/companysearch.html
* Uses NASDAQ IPO lists
 * https://www.nasdaq.com/markets/ipos/
* Skips ambiguous company names (do manually)
* Takes latest file if multiple files are available (S-1, S-1/A etc.)

In [1]:
import edgar

import pandas as pd

from pathlib import Path

import datetime
from datetime import timedelta
from collections import OrderedDict

In [72]:
# params
date_range = ["2010-01-01", datetime.datetime.today().strftime('%Y-%m-%d')]
start, end = [datetime.datetime.strptime(_, "%Y-%m-%d") for _ in date_range]
date_dict = OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m"), None) for _ in range((end - start).days)).keys()
print('date_dict:', date_dict)

date_dict: odict_keys(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06', '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12', '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06', '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12', '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05'

### Scrape NASDAQ IPO Lists

In [73]:
df_symbols = pd.DataFrame()

for x in date_dict:
    df_symbols = df_symbols.append(pd.read_html('https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=' + x)[0], ignore_index=True)
    
df_symbols.index = df_symbols['Symbol']        
df_symbols.drop_duplicates('Symbol', inplace=True)
df_symbols.dropna(inplace=True, subset=['Symbol'])

C:\Users\Void\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [74]:
df_symbols.head(3)

,Company Name,Date Priced,Market,Offer Amount,Price,Shares,Symbol,0,1,2
Symbol,,,,,,,,,,
CELM,"CHINA ELECTRIC MOTOR, INC.",1/29/2010,NASDAQ,"$22,500,000",$4.50,5000000,CELM,NaN,NaN,NaN
CTC,IFM INVESTMENTS LTD,1/28/2010,New York Stock Exchange,"$87,412,500",$7,12487500,CTC,NaN,NaN,NaN
AMCF,ANDATEE CHINA MARINE FUEL SERVICES CORP,1/26/2010,NASDAQ,"$19,750,002",$6.30,3134921,AMCF,NaN,NaN,NaN


### Batch Download

In [75]:
#batch download
counter = 0

for x in df_symbols['Symbol']:
    try:
        counter += 1
        print('\n( ' + str(counter) + ' / ' + str(df_symbols.shape[0]) + ' ) ' + str(x))

        #check if exists
        if Path("./Data/" + x + ".htm").is_file():
            print(x + ' data already exists, skipping...')
            continue
        # file exists

        #create company
        tmpEdgar = edgar.Edgar()
        possible_companies = tmpEdgar.findCompanyName(df_symbols.loc[x]['Company Name'])
        print('possible_companies:', possible_companies)

        #validate
        if len(possible_companies) == 0:
            print('no possible companies:', x)
            continue

        name = possible_companies[0]
        cik = tmpEdgar.getCikByCompanyName(possible_companies[0])
        company = edgar.Company(name, cik)

        #look for S-1
        tree = company.getAllFilings(filingType = "S-1")
        docs = edgar.getDocuments(tree, noOfDocuments=5)

        #if multiple amendments exist
        #filter ones with length lower than 500K (statistically the mean is higher)
        docs = [d for d in docs if len(d) > 500000]

        #write file
        if len(docs) > 0:
            with open("./Data/" + x + ".htm", "w", encoding="utf-8") as f:
                f.write(docs[0].decode("utf-8"))
                print('Got data for ' + x + ' chose ' + str(len(docs[0])))
    except Exception as e:
        print(x, e)


( 1 / 1697 ) CELM
possible_companies: ['CHINA ELECTRIC MOTOR, INC.']
doc 3369247 https://www.sec.gov/Archives/edgar/data/1421526/000114420410014081/v177360_s1a.htm
doc 3362600 https://www.sec.gov/Archives/edgar/data/1421526/000114420410011462/v175718_s1.htm
doc 4030350 https://www.sec.gov/Archives/edgar/data/1421526/000114420410003731/v172338_s1a.htm
doc 4021320 https://www.sec.gov/Archives/edgar/data/1421526/000114420410002573/v171635_s1a.htm
doc 4009585 https://www.sec.gov/Archives/edgar/data/1421526/000114420410001427/v170522_s1a.htm
Got data for CELM chose 3369247

( 2 / 1697 ) CTC
possible_companies: ['IFM INVESTMENTS LTD']

( 3 / 1697 ) AMCF
possible_companies: ['ANDATEE CHINA MARINE FUEL SERVICES CORP']
doc 72613 https://www.sec.gov/Archives/edgar/data/1469606/000114420410003618/v172218_s1mef.htm
doc 72607 https://www.sec.gov/Archives/edgar/data/1469606/000114420410003496/v172218_s1.htm
doc 2313146 https://www.sec.gov/Archives/edgar/data/1469606/000114420410002541/v171701_s1a.h

doc 2143852 https://www.sec.gov/Archives/edgar/data/1410098/000114420409067091/v169969_s1a.htm
doc 1976182 https://www.sec.gov/Archives/edgar/data/1410098/000114420409062141/v167363_s1.htm
Got data for CRMD chose 2014757

( 21 / 1697 ) FIBK
possible_companies: ['FIRST INTERSTATE BANCSYSTEM INC', 'FIRST INTERSTATE BANCSYSTEM OF MONTANA INC', 'FIRST INTERSTATE BANCSYSTEM, INC.']
doc 30472 https://www.sec.gov/Archives/edgar/data/860413/000095012310027397/m55325sv1mef.htm
doc 3036000 https://www.sec.gov/Archives/edgar/data/860413/000095012310026955/c55325a3sv1za.htm
doc 3060513 https://www.sec.gov/Archives/edgar/data/860413/000095012310023119/c55325a2sv1za.htm
doc 3144089 https://www.sec.gov/Archives/edgar/data/860413/000095012310020091/c55325a1sv1za.htm
doc 3672366 https://www.sec.gov/Archives/edgar/data/860413/000095012310003037/c55325sv1.htm
Got data for FIBK chose 3036000

( 22 / 1697 ) CALX
possible_companies: ['CALIX, INC']
doc 2169125 https://www.sec.gov/Archives/edgar/data/1406666/

doc 2027181 https://www.sec.gov/Archives/edgar/data/1267602/000095012310035507/g20643a5sv1za.htm
Got data for ALIM chose 2280200

( 39 / 1697 ) DHRM
possible_companies: ['LIANLUO SMART LTD']
doc 81858 https://www.sec.gov/Archives/edgar/data/1474627/000119312510068490/ds1a.htm
doc 1441653 https://www.sec.gov/Archives/edgar/data/1474627/000119312510063278/ds1a.htm
doc 1404473 https://www.sec.gov/Archives/edgar/data/1474627/000119312510055189/ds1a.htm
doc 1396409 https://www.sec.gov/Archives/edgar/data/1474627/000119312510046847/ds1a.htm
doc 1391522 https://www.sec.gov/Archives/edgar/data/1474627/000119312510032302/ds1a.htm
Got data for DHRM chose 1441653

( 40 / 1697 ) DVOX
possible_companies: ['DYNAVOX INC.']
doc 2867285 https://www.sec.gov/Archives/edgar/data/1479426/000104746910003510/a2196211zs-1a.htm
doc 2871012 https://www.sec.gov/Archives/edgar/data/1479426/000104746910003268/a2197844zs-1a.htm
doc 2793871 https://www.sec.gov/Archives/edgar/data/1479426/000104746910002497/a2196211z


( 56 / 1697 ) RRTS
possible_companies: ['ROADRUNNER TRANSPORTATION SYSTEMS, INC.']
doc 1526290 https://www.sec.gov/Archives/edgar/data/1440024/000095012310047107/c55423a8sv1za.htm
doc 1528109 https://www.sec.gov/Archives/edgar/data/1440024/000095012310046147/c55423a7sv1za.htm
doc 1531194 https://www.sec.gov/Archives/edgar/data/1440024/000095012310040997/c55423a6sv1za.htm
doc 1548134 https://www.sec.gov/Archives/edgar/data/1440024/000095012310037406/c55423a5sv1za.htm
doc 1456420 https://www.sec.gov/Archives/edgar/data/1440024/000095012310031901/c55423a4sv1za.htm
Got data for RRTS chose 1526290

( 57 / 1697 ) TNAV
possible_companies: ['TELENAV, INC.']
doc 2328065 https://www.sec.gov/Archives/edgar/data/1474439/000119312510117975/ds1a.htm
doc 2328087 https://www.sec.gov/Archives/edgar/data/1474439/000119312510117701/ds1a.htm
doc 2329019 https://www.sec.gov/Archives/edgar/data/1474439/000119312510116372/ds1a.htm
doc 2308461 https://www.sec.gov/Archives/edgar/data/1474439/00011931251009336

doc 3146189 https://www.sec.gov/Archives/edgar/data/1374310/000104746910005611/a2197704zs-1a.htm
doc 3154871 https://www.sec.gov/Archives/edgar/data/1374310/000104746910005418/a2197704zs-1a.htm
doc 2203058 https://www.sec.gov/Archives/edgar/data/1374310/000104746910003625/a2197704zs-1a.htm
doc 2157940 https://www.sec.gov/Archives/edgar/data/1374310/000104746910002050/a2197106zs-1.htm
Got data for CBOE chose 3146189

( 74 / 1697 ) BORN
possible_companies: ['CHINA NEW BORUN CORP']

( 75 / 1697 ) CORN
possible_companies: ['TEUCRIUM COMMODITY TRUST']
doc 1447177 https://www.sec.gov/Archives/edgar/data/1471824/000165495418004184/tags_322181045am.htm
doc 1375993 https://www.sec.gov/Archives/edgar/data/1471824/000165495418004182/cane_322181037am.htm
doc 1325369 https://www.sec.gov/Archives/edgar/data/1471824/000165495418004180/soyb_32118125pm.htm
doc 1347285 https://www.sec.gov/Archives/edgar/data/1471824/000165495418003188/tags_322181045am.htm
doc 1305345 https://www.sec.gov/Archives/edgar/d

...

doc 2335073 https://www.sec.gov/Archives/edgar/data/1698113/000121390017010952/fs12017_legacyacq.htm
Got data for LGC'U chose 2372130

( 1560 / 1697 ) SFIX
possible_companies: ['STITCH FIX, INC.']
doc 1582185 https://www.sec.gov/Archives/edgar/data/1576942/000119312517333497/d400510ds1a.htm
doc 1559691 https://www.sec.gov/Archives/edgar/data/1576942/000119312517313629/d400510ds1.htm
Got data for SFIX chose 1582185

( 1561 / 1697 ) JT
possible_companies: ['JIANPU TECHNOLOGY INC.']

( 1562 / 1697 ) ASNS
possible_companies: ['ARSANIS, INC.']
doc 2674550 https://www.sec.gov/Archives/edgar/data/1501697/000119312517343803/d427310ds1a.htm
doc 2662368 https://www.sec.gov/Archives/edgar/data/1501697/000119312517333514/d427310ds1a.htm
doc 2648847 https://www.sec.gov/Archives/edgar/data/1501697/000119312517315505/d427310ds1.htm
Got data for ASNS chose 2674550

( 1563 / 1697 ) SEND
possible_companies: ['SENDGRID, INC.']
doc 90094 https://www.sec.gov/Archives/edgar/data/1477425/000110465918022718/a

doc 3373702 https://www.sec.gov/Archives/edgar/data/1690680/000119312517360780/d298983ds1a.htm
doc 3336557 https://www.sec.gov/Archives/edgar/data/1690680/000119312517359952/d298983ds1a.htm
Got data for NMRK chose 3424081

( 1587 / 1697 ) GIG'U
possible_companies: ['GIGCAPITAL, INC.']
doc 36900 https://www.sec.gov/Archives/edgar/data/1719489/000119312517364429/d476098ds1mef.htm
doc 1144534 https://www.sec.gov/Archives/edgar/data/1719489/000119312517354167/d471396ds1a.htm
doc 1140238 https://www.sec.gov/Archives/edgar/data/1719489/000119312517344053/d471396ds1.htm
Got data for GIG'U chose 1144534

( 1588 / 1697 ) DNLI
possible_companies: ['DENALI THERAPEUTICS INC.']
doc 2317525 https://www.sec.gov/Archives/edgar/data/1714899/000119312517363254/d445892ds1a.htm
doc 2317656 https://www.sec.gov/Archives/edgar/data/1714899/000119312517351618/d445892ds1a.htm
doc 2298446 https://www.sec.gov/Archives/edgar/data/1714899/000119312517340997/d445892ds1.htm
Got data for DNLI chose 2317525

( 1589 / 

doc 4457462 https://www.sec.gov/Archives/edgar/data/1717307/000104746917007408/a2233965zs-11a.htm
doc 4453823 https://www.sec.gov/Archives/edgar/data/1717307/000104746917007236/a2233892zs-11.htm
Got data for ILPT chose 4512961

( 1610 / 1697 ) LBRT
possible_companies: ['LIBERTY OILFIELD SERVICES INC.']
doc 33614 https://www.sec.gov/Archives/edgar/data/1694028/000119312518009270/d523240ds1mef.htm
doc 1699731 https://www.sec.gov/Archives/edgar/data/1694028/000119312518000719/d313047ds1a.htm
doc 1693050 https://www.sec.gov/Archives/edgar/data/1694028/000119312517370996/d313047ds1a.htm
doc 1665943 https://www.sec.gov/Archives/edgar/data/1694028/000119312517338194/d313047ds1a.htm
doc 1675808 https://www.sec.gov/Archives/edgar/data/1694028/000119312517262436/d313047ds1a.htm
Got data for LBRT chose 1699731

( 1611 / 1697 ) NEBUU
possible_companies: ['NEBULA ACQUISITION CORP']
doc 2142050 https://www.sec.gov/Archives/edgar/data/1720353/000121390018000280/fs12018a3b_nebulaacq.htm
doc 2148339 ht


( 1632 / 1697 ) IPIC
possible_companies: ['IPIC ENTERTAINMENT INC.']

( 1633 / 1697 ) SLGL
possible_companies: ['SOL-GEL TECHNOLOGIES LTD.']

( 1634 / 1697 ) VICI
possible_companies: ['VICI PROPERTIES INC.']
doc 1022687 https://www.sec.gov/Archives/edgar/data/1705696/000119312518159401/d542153ds11.htm
doc 35905 https://www.sec.gov/Archives/edgar/data/1705696/000119312518027632/d466469ds11mef.htm
doc 2492453 https://www.sec.gov/Archives/edgar/data/1705696/000119312518024362/d496153ds11a.htm
doc 80150 https://www.sec.gov/Archives/edgar/data/1705696/000119312518022381/d496153ds11a.htm
doc 2520191 https://www.sec.gov/Archives/edgar/data/1705696/000119312518015216/d496153ds11a.htm
Got data for VICI chose 1022687

( 1635 / 1697 ) GNPX
possible_companies: ['GENPREX, INC.']
doc 1253758 https://www.sec.gov/Archives/edgar/data/1595248/000119312518194621/d591407ds1a.htm
doc 1252973 https://www.sec.gov/Archives/edgar/data/1595248/000119312518169788/d591407ds1.htm
doc 1374871 https://www.sec.gov/A

possible_companies: ['MORPHOSYS AG']

( 1662 / 1697 ) EAF
EAF data already exists, skipping...

( 1663 / 1697 ) AGMH
possible_companies: ['AGM GROUP HOLDINGS, INC.']

( 1664 / 1697 ) ZUO
ZUO data already exists, skipping...

( 1665 / 1697 ) SPOT
possible_companies: ['SPOTIFY TECHNOLOGY S.A.']

( 1666 / 1697 ) HJLI
HJLI data already exists, skipping...

( 1667 / 1697 ) TDACU
TDACU data already exists, skipping...

( 1668 / 1697 ) ITRM
ITRM data already exists, skipping...

( 1669 / 1697 ) KNSA
KNSA data already exists, skipping...

( 1670 / 1697 ) SRRK
SRRK data already exists, skipping...

( 1671 / 1697 ) GSKY
GSKY data already exists, skipping...

( 1672 / 1697 ) CLPS
possible_companies: ['CLPS INC']

( 1673 / 1697 ) EVOP
EVOP data already exists, skipping...

( 1674 / 1697 ) PS
PS data already exists, skipping...

( 1675 / 1697 ) VTIQU
VTIQU data already exists, skipping...

( 1676 / 1697 ) TMCXU
TMCXU data already exists, skipping...

( 1677 / 1697 ) HUYA
possible_companies: ['HUYA 